In [1]:
# default_exp core

# ParseDeep

> Parse IEX DEEP files

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
from pcap import pcap
from struct import unpack, unpack_from, calcsize
from functools import cache

In [4]:
#hide
import pyarrow as pa 
import pyarrow.parquet as pq
import pandas as pd

In [5]:
#export
class Msg():
    "Class for working with a DEEP message"
    
    msg_type = None
    msg_sub_type = None
    
    def __init__(self, sequence_number, msg, debug=False):
        self.sequence_number = sequence_number
        self.type = msg[0:1]
        self.debug = debug
        if self.debug:
            self.bytes = msg
            
    @classmethod
    def matches_msg_type(cls, msg):
        return msg[0:1] == cls.msg_type
    
    @classmethod
    def matches_msg_sub_type(cls, msg):
        return True # override message type has subtypes
    
    @classmethod
    @cache
    def type_clss(cls):
        all = []
        for type_cls in cls.__subclasses__():
            all.append(type_cls)
            for sub_type_cls in type_cls.__subclasses__():
                all.append(sub_type_cls)
        return all
            
    @classmethod
    def factory(cls, sequence_number, msg, debug=False):
        for type_cls in Msg.type_clss():
            if type_cls.matches_msg_type(msg) and type_cls.matches_msg_sub_type(msg):
                return type_cls(sequence_number, msg, debug=debug)
        return UnsupportedMsg(sequence_number, msg, debug=debug)

In [6]:
#export
class PriceLevelUpdateMsg(Msg):
    "Class for working with a Price Level Update message"

    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<bq8siq"    
        
        self.event_flags, self.timestamp, self.symbol, self.size, self.price = \
            unpack_from(fmt, msg[1:])

        self.symbol = self.symbol.rstrip().decode('UTF-8')
        self.price = self.price / 10000
                
        super().__init__(sequence_number, msg, debug=debug)        

In [7]:
#export
class BuySidePriceLevelUpdageMsg(PriceLevelUpdateMsg):
    "Class for working with a Buy Side Price Level Update message"
    msg_type = b'8'

In [8]:
#export
class SellSidePriceLevelUpdageMsg(PriceLevelUpdateMsg):
    "Class for working with a Sell Side Price Level Update message"
    msg_type = b'5'

In [9]:
#export
class SystemEventMsg(Msg):
    "Class for working with a System Event message"
    
    @classmethod
    def matches_msg_sub_type(cls, msg):
        return msg[1:2] == cls.msg_sub_type

In [10]:
#export
class StartOfMessagesMsg(SystemEventMsg):
    "Class for working with Start of Messages message"
    msg_type = b'S'
    msg_sub_type = b'O'

In [11]:
#export
class StartOfSystemHoursMsg(SystemEventMsg):
    "Class for working with Start of System Hours message"
    msg_type = b'S'
    msg_sub_type = b'S'

In [12]:
#export
class StartOfRegularMarketHoursMsg(SystemEventMsg):
    "Class for working with Start of Regular Market Hours message"
    msg_type = b'S'
    msg_sub_type = b'R'

In [13]:
#export
class EndOfRegularMarketHoursMsg(SystemEventMsg):
    "Class for working with End of Regular Market Hours message"
    msg_type = b'S'
    msg_sub_type = b'M'

In [14]:
#export
class EndOfSystemHoursMsg(SystemEventMsg):
    "Class for working with End of System Hours message"
    msg_type = b'S'
    msg_sub_type = b'E'

In [15]:
#export
class EndOfMessagesMsg(SystemEventMsg):
    "Class for working with End of Messages message"
    msg_type = b'S'
    msg_sub_type = b'C'

In [16]:
#export
class ShortSalePriceTestStatusMsg(Msg):
    "Class for working with Short Sale Price Test Status message"
  
    @classmethod
    def matches_msg_sub_type(cls, msg):
        return unpack("<b", msg[1:2])[0] == cls.msg_sub_type
        
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<bq8sb"    
        
        self.status, self.timestamp, self.symbol, self.detail = \
            unpack_from(fmt, msg[1:])

        self.symbol = self.symbol.rstrip().decode('UTF-8')
                
        super().__init__(sequence_number, msg, debug=debug)

In [17]:
#export
class ShortSalePriceTestNotInEffect(ShortSalePriceTestStatusMsg):
    "Class for working with Short Sale Price Test Not In Effect message"
    msg_type = b'P'
    msg_sub_type = 0

In [18]:
#export
class ShortSalePriceTestInEffect(ShortSalePriceTestStatusMsg):
    "Class for working with Short Sale Price Test In Effect message"
    msg_type = b'P'
    msg_sub_type = 1

In [19]:
#export
class OperationalHaltStatusMsg(Msg):
    "Class for working with Operational Halt Status message"
    
    @classmethod
    def matches_msg_sub_type(cls, msg):
        return unpack("<c", msg[1:2])[0] == cls.msg_sub_type

    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<bq8s"    
        
        self.status, self.timestamp, self.symbol = \
            unpack_from(fmt, msg[1:])

        self.symbol = self.symbol.rstrip().decode('UTF-8')
            
        super().__init__(sequence_number, msg, debug=debug)

In [20]:
#export
class IEXSpecificOperationalTradingHaltMsg(OperationalHaltStatusMsg):
    "Class for working with IEX Specific Operational Trading Halt message"
    msg_type = b'O'
    msg_sub_type = b'O'

In [21]:
#export
class NotOperationallyHaltedOnIEXMsg(OperationalHaltStatusMsg):
    "Class for working with Not Operationally Halted On IEX message"
    msg_type = b'O'
    msg_sub_type = b'N'

In [22]:
#export
class TradingStatusMsg(Msg):
    "Class for working with Trading Status message"

    @classmethod
    def matches_msg_sub_type(cls, msg):
        return unpack("<c", msg[1:2])[0] == cls.msg_sub_type
    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<cq12s"    
        
        self.status, self.timestamp, s = \
            unpack_from(fmt, msg[1:])

        self.symbol, self.reason = s[0:7], s[8:11]
        
        self.symbol = self.symbol.rstrip().decode('UTF-8')
        self.reason = self.reason.rstrip().decode('UTF-8')

        super().__init__(sequence_number, msg, debug=debug)

In [23]:
#export
class TradingHaltedAcrossAllUSEquityMarketsMsg(TradingStatusMsg):
    "Class for working with Trading Haldted Across All US Equity Markets message"
    msg_type = b'H'
    msg_sub_type = b'H'  

In [24]:
#export
class TradingHaltReleasedIntoAnOrderAcceptancePeriodOnIEXMsg(TradingStatusMsg):
    "Class for working with Trading Halt Released Into An Order Acceptance Period On IEX message"
    msg_type = b'H'
    msg_sub_type = b'O'

In [25]:
#export
class TradingPausedAndOrderAcceptancePeriodOnIEXMsg(TradingStatusMsg):
    "Class for working with Trading Paused And Order Acceptance Period On IEX message"
    msg_type = b'H'
    msg_sub_type = b'P'

In [26]:
#export
class TradingOnIEXMsg(TradingStatusMsg):
    "Class for working with Trading On IEX message"
    msg_type = b'H'
    msg_sub_type = b'T'

In [27]:
#export
class SecurityDirectoryMsg(Msg):
    "Class for working with Security Directory message"
    msg_type = b'D'
    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<bq8siqb"    
        
        self.flags, self.timestamp, self.symbol, self.round_lot_size, self.adjusted_poc_price, self.luld_tier = \
            unpack_from(fmt, msg[1:])
        
        self.symbol = self.symbol.rstrip().decode('UTF-8')
        self.adjusted_poc_price = self.adjusted_poc_price / 10000
        
        super().__init__(sequence_number, msg, debug=debug)

In [28]:
#export
class TradeReportMsg(Msg):
    "Class for working with Trade Report message"
    msg_type = b'T'
    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<bq8siqq"    
        
        self.flags, self.timestamp, self.symbol, self.size, self.price, self.trade_id = \
            unpack_from(fmt, msg[1:])
        
        self.symbol = self.symbol.rstrip().decode('UTF-8')
        self.price = self.price / 10000
        
        super().__init__(sequence_number, msg, debug=debug)

In [29]:
#export
class AuctionInformationMsg(Msg):
    "Class for working with Auction Information message"

    @classmethod
    def matches_msg_sub_type(cls, msg):
        return unpack("<c", msg[1:2])[0] == cls.msg_sub_type
    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<cq8siqqibbiqqqq" 
        
        self.type, self.timestamp, self.symbol, self.paired_shares, self.reference_price, \
            self.indivitive_clearing_price, self.imbalance_shares, self.imbalance_side, \
            self.extension_number, self.scheduled_auction_time, self.auction_book_clearing_price, \
            self.collar_reference_price, self.lower_auction_collar, self.upper_auction_collar = \
            unpack_from(fmt, msg[1:])

        self.symbol = self.symbol.rstrip().decode('UTF-8')
        self.reference_price = self.reference_price / 10000
        self.indivitive_clearing_price = self.indivitive_clearing_price / 10000
        self.auction_book_clearing_price = self.auction_book_clearing_price / 10000
        self.collar_reference_price = self.collar_reference_price / 10000
        self.lower_auction_collar = self.lower_auction_collar / 10000
        self.upper_auction_collar = self.upper_auction_collar / 10000
     
        super().__init__(sequence_number, msg, debug=debug)

In [30]:
#export
class OpeningAuctionMsg(AuctionInformationMsg):
    "Class for working with Opening Auction message"
    msg_type = b'A'
    msg_sub_type = b'O'

In [31]:
#export
class ClosingAuctionMsg(AuctionInformationMsg):
    "Class for working with Closing Auction message"
    msg_type = b'A'
    msg_sub_type = b'C'

In [32]:
#export
class IPOAuctionMsg(AuctionInformationMsg):
    "Class for working with IPO Auction message"
    msg_type = b'A'
    msg_sub_type = b'I'

In [33]:
#export
class HaltAuctionMsg(AuctionInformationMsg):
    "Class for working with Halt Auction message"
    msg_type = b'A'
    msg_sub_type = b'H'

In [34]:
#export
class VolatilityAuctionMsg(AuctionInformationMsg):
    "Class for working with Volatility Auction message"
    msg_type = b'A'
    msg_sub_type = b'V'

In [35]:
#export
class SecurityEventMsg(Msg):
    "Class for working with Security Event message"

    @classmethod
    def matches_msg_sub_type(cls, msg):
        return unpack("<c", msg[1:2])[0] == cls.msg_sub_type
    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<cq8s" 
        
        self.type, self.timestamp, self.symbol = \
            unpack_from(fmt, msg[1:])

        self.symbol = self.symbol.rstrip().decode('UTF-8')
     
        super().__init__(sequence_number, msg, debug=debug)

In [36]:
#export
class OpeningProcessCompleteMsg(SecurityEventMsg):
    "Class for working with Opening Process Complete message"
    msg_type = b'E'
    msg_sub_type = b'O'

In [37]:
#export
class ClosingProcessCompleteMsg(SecurityEventMsg):
    "Class for working with Closing Process Complete message"
    msg_type = b'E'
    msg_sub_type = b'C'

In [ ]:
#export
class OfficialPriceMsg(Msg):
    "Class for working with Official Price message"

    @classmethod
    def matches_msg_sub_type(cls, msg):
        return unpack("<c", msg[1:2])[0] == cls.msg_sub_type
    
    def __init__(self, sequence_number, msg, debug=False):
        fmt = "<cq8sq" 
        
        self.type, self.timestamp, self.symbol, self.official_price = \
            unpack_from(fmt, msg[1:])

        self.symbol = self.symbol.rstrip().decode('UTF-8')
        self.official_price = self.official_price / 10000
     
        super().__init__(sequence_number, msg, debug=debug)

In [ ]:
#export
class IEXOfficialOpeningPriceMsg(OfficialPriceMsg):
    "Class for working with IEX Official Opening Price message"
    msg_type = b'X'
    msg_sub_type = b'Q'

In [ ]:
#export
class IEXOfficialClosingPriceMsg(OfficialPriceMsg):
    "Class for working with IEX Official Closing Price message"
    msg_type = b'X'
    msg_sub_type = b'M'

In [38]:
#export
class UnsupportedMsg(Msg):
    "Class for catching unsupported messages"

In [39]:
#export
class Pkt():
    "Class for working with a DEEP packet"
    def __init__(self, pkt, debug=False):
        self.debug = debug
        if self.debug:
            self.bytes = pkt
        
        self.index = -1
        
        self.version     = b"\x01"
        self.reserved    = b"\x00"
        self.protocol_id = b"\x04\x80"        
        self.channel_id  = b"\x01\x00\x00\x00"
            
        header = (
            self.version + 
            self.reserved + 
            self.protocol_id + 
            self.channel_id
        )
    
        pkt = pkt[pkt.find(header) + len(header):]

        fmt = "<lhhqqq"
        
        self.session_id, self.payload_len, self.msg_count, self.stream_offset, self.first_msg_seq_num, self.send_time = \
            unpack_from(fmt, pkt)
                  
        self.msgs = pkt[calcsize(fmt):]        

    def next_msg(self):
        self.index += 1
        if self.index == self.msg_count:
            return None
        else:
            frag, msgs = self.msgs[0:2], self.msgs[2:]            
            msg_len = unpack("<h", frag)[0]
            msg, self.msgs = msgs[0:msg_len], msgs[msg_len:]
            return Msg.factory(self.first_msg_seq_num + self.index, msg, debug=self.debug)

In [40]:
#export 
class Deep():
    "Class for working with a DEEP export file"
    def __init__(self, path, debug=False):
        self._pkt = None
        self.pcap = pcap(path)
        self.debug = debug
        
    def __set_pkt(self, ts, _pkt):
        self._pkt = Pkt(_pkt, debug=self.debug) 
        
    def next_pkt(self):
        if self.pcap.dispatch(1, self.__set_pkt) == 1:
            return(self._pkt) 
        else:
            return None

In [41]:
deep = Deep('input\data_feeds_20210924_20210924_IEXTP1_DEEP1.0.pcap', debug=True)

In [42]:
n = 0
while pkt := deep.next_pkt():
    while msg := pkt.next_msg():
        if type(msg) == UnsupportedMsg:
            print(vars(msg))
            raise

{'sequence_number': 148606, 'type': b'X', 'debug': True, 'bytes': b"XQ\xbaYt\x060\xc5\xa7\x16ZIEXT   \x10'\x00\x00\x00\x00\x00\x00"}


RuntimeError: No active exception to reraise

In [ ]:
deep = Deep('input\data_feeds_20210924_20210924_IEXTP1_DEEP1.0.pcap')

In [ ]:
chunk_size = 1_000_000

In [ ]:
def write_chunk(msgs):
    df = pd.DataFrame({
        'flags': [m.event_flags for m in msgs],
        'timestamp': [m.timestamp for m in msgs],
        'symbol': [m.symbol for m in msgs],
        'size': [m.size for m in msgs],
        'price': [m.price for m in msgs],
        'side': ['buy' if type(msg) == BuySidePriceLevelUpdageMsg else 'sell' for m in msgs]
    })
    table = pa.Table.from_pandas(df)
    pq.write_table(table, 'output\price_level_updates_{}.parquet'.format(msgs[0].timestamp))

In [ ]:
i = 1
msgs = []
while pkt := deep.next_pkt():
    while msg := pkt.next_msg():
        if type(msg) != UnsupportedMsg:
            msgs.append(msg)                    
        if len(msgs) >= chunk_size:
            write_chunk(msgs)
            msgs = []
            i += 1
            print(".", end="")

# write remaining less than chunk_size left overs
write_chunk(msgs)
print(".")